In [1]:
#Se importan las librerias 

import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time 
import requests
import stylecloud
import stop_words
import urllib.request
from IPython.display import Image
import nltk
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import pandas.util.testing as tm
nltk.download('punkt')
import string
import re
nltk.download('stopwords')
sns.set(rc={'figure.figsize':(11.7,8.27)})

/anaconda3/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rodolfopardo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rodolfopardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
tuits = pd.read_csv('tuits_Es.csv')

In [4]:
tuits.head()

,Vertex 1,Relationship,Tweet,URLs in Tweet,Domains in Tweet,Hashtags in Tweet,Date,Time,Twitter Page for Tweet,Latitude,Longitude,Favorite Count,Language
0,bexxmodd,Retweet,A new #GoogleCloud data analytics tool is in t...,https://www.crn.com/news/cloud/a-new-google-cl...,crn.com,googlecloud,10/5/2021,10:02:09,https://twitter.com/bexxmodd/status/1391695045...,NaN,NaN,0,en
1,bmind_es,Tweet,🗓️El próximo 12 de mayo nos vemos en #GoogleCl...,https://googlecloudhexagon.com/registro.html?%...,googlecloudhexagon.com,googlecloudhexagon googlecloud gcp ia,10/5/2021,9:33:41,https://twitter.com/bmind_es/status/1391687877...,NaN,NaN,0,es
2,1martaalvarez1,Retweet,🗓️El próximo 12 de mayo nos vemos en #GoogleCl...,https://googlecloudhexagon.com/registro.html?%...,googlecloudhexagon.com,googlecloudhexagon googlecloud gcp ia,10/5/2021,10:54:45,https://twitter.com/1martaalvarez1/status/1391...,NaN,NaN,0,es
3,bareivigneron,Tweet,"Avec des offres étendues sur #GoogleCloud, les...",https://www.teradata.fr/Press-Releases/2021/Te...,teradata.fr,googlecloud teradata vantage cloud,10/5/2021,19:33:29,https://twitter.com/bareivigneron/status/13918...,NaN,NaN,1,fr
4,paradigmate,Mentions,🎧 La filosofía #DevOps triunfa en muchas empre...,NaN,NaN,devops podcast googlecloud,10/5/2021,15:54:00,https://twitter.com/paradigmate/status/1391783...,NaN,NaN,4,es


In [5]:
tuits.columns

Index(['Vertex 1', 'Relationship', 'Tweet', 'URLs in Tweet',
       'Domains in Tweet', 'Hashtags in Tweet', 'Date', 'Time',
       'Twitter Page for Tweet', 'Latitude', 'Longitude', 'Favorite Count',
       'Language'],
      dtype='object')

In [6]:
columnas = ['usuario', 'relacion', 'tuit', 'url', 'dominio', 'hashtag', 'date', 'time', 'page', 'latitud', 'longitud',
           'favoritos', 'language']

tuits.columns = columnas

In [8]:
tuits.head()

,usuario,relacion,tuit,url,dominio,hashtag,date,time,page,latitud,longitud,favoritos,language
0,bexxmodd,Retweet,A new #GoogleCloud data analytics tool is in t...,https://www.crn.com/news/cloud/a-new-google-cl...,crn.com,googlecloud,10/5/2021,10:02:09,https://twitter.com/bexxmodd/status/1391695045...,NaN,NaN,0,en
1,bmind_es,Tweet,🗓️El próximo 12 de mayo nos vemos en #GoogleCl...,https://googlecloudhexagon.com/registro.html?%...,googlecloudhexagon.com,googlecloudhexagon googlecloud gcp ia,10/5/2021,9:33:41,https://twitter.com/bmind_es/status/1391687877...,NaN,NaN,0,es
2,1martaalvarez1,Retweet,🗓️El próximo 12 de mayo nos vemos en #GoogleCl...,https://googlecloudhexagon.com/registro.html?%...,googlecloudhexagon.com,googlecloudhexagon googlecloud gcp ia,10/5/2021,10:54:45,https://twitter.com/1martaalvarez1/status/1391...,NaN,NaN,0,es
3,bareivigneron,Tweet,"Avec des offres étendues sur #GoogleCloud, les...",https://www.teradata.fr/Press-Releases/2021/Te...,teradata.fr,googlecloud teradata vantage cloud,10/5/2021,19:33:29,https://twitter.com/bareivigneron/status/13918...,NaN,NaN,1,fr
4,paradigmate,Mentions,🎧 La filosofía #DevOps triunfa en muchas empre...,NaN,NaN,devops podcast googlecloud,10/5/2021,15:54:00,https://twitter.com/paradigmate/status/1391783...,NaN,NaN,4,es


In [10]:
tuits = tuits[tuits['language'] == "es"]

In [14]:
tuits.relacion.value_counts()

Mentions             93
MentionsInRetweet    70
Retweet              50
Replies to           48
Tweet                28
Name: relacion, dtype: int64

In [20]:
tuits.dominio.value_counts().head()

withgoogle.com     20
google.com         19
hopin.com          17
innovaspain.com    16
ilunion.com        16
Name: dominio, dtype: int64

In [21]:
lista = tuits.tuit

In [23]:
with open('titulo.txt', 'w') as f:
    for item in lista:
        f.write("%s\n" % item)

In [25]:
string = str(lista)
stop = stop_words.get_stop_words("es")
stylecloud.gen_stylecloud(string,
                          icon_name='fab fa-twitter',
                          custom_stopwords=stop, 
                         output_name='titulo_tw_Es.png')

In [26]:
influencers = pd.read_csv('Influencers_es.csv')

In [28]:
influencers.head()

,Vertex,Name,Followed,Followers,Tweets,Favorites,Description,Location,Web,Joined Twitter Date (UTC),...,Custom Menu Item Text,Custom Menu Item Action,Profile Banner Url,Default Profile,Default Profile Image,Geo Enabled,Language,Listed Count,Profile Background Image Url,Verified
0,bexxmodd,"Beka ""Bexx"" Modebade",5947,8496,848213,141717,My opinions are a byproduct of biases introduc...,"Maryland, USA",https://t.co/5qHf4jSZ8t,12/9/2015 1:17,...,Open Twitter Page for This Person,https://twitter.com/bexxmodd,https://pbs.twimg.com/profile_banners/35326553...,False,False,True,NaN,79,https://abs.twimg.com/images/themes/theme15/bg...,False
1,gehring__chris,Christian Gehring,135,428,555,279,Cloud expert. Excited to be a part of a team d...,Germany,https://t.co/GBF8xuZmxS,11/6/2013 7:07,...,Open Twitter Page for This Person,https://twitter.com/gehring__chris,https://pbs.twimg.com/profile_banners/15006097...,False,False,False,NaN,6,https://abs.twimg.com/images/themes/theme15/bg...,False
2,bmind_es,BMIND,136,179,263,61,Somos una consultora que añade valor al negoci...,"Madrid, Comunidad de Madrid",https://t.co/yxILNx6e4n,11/10/2015 20:15,...,Open Twitter Page for This Person,https://twitter.com/bmind_es,https://pbs.twimg.com/profile_banners/39306770...,True,False,True,NaN,6,https://abs.twimg.com/images/themes/theme1/bg.png,False
3,1martaalvarez1,Marta Alvarez,230,244,1177,606,NaN,Madrid,NaN,12/4/2011 9:04,...,Open Twitter Page for This Person,https://twitter.com/1martaalvarez1,https://pbs.twimg.com/profile_banners/28092800...,False,False,False,NaN,7,https://abs.twimg.com/images/themes/theme14/bg...,False
4,bareivigneron,Séverine VIGNERON,170,105,1355,1168,NaN,NaN,NaN,7/6/2010 11:37,...,Open Twitter Page for This Person,https://twitter.com/bareivigneron,NaN,True,False,True,NaN,0,https://abs.twimg.com/images/themes/theme1/bg.png,False


In [29]:
influencers.Verified.value_counts()

False    207
True      57
Name: Verified, dtype: int64

In [35]:
influencers.sort_values(by = "Followers", ascending = False).head(10)

,Vertex,Name,Followed,Followers,Tweets,Favorites,Description,Location,Web,Joined Twitter Date (UTC),...,Custom Menu Item Text,Custom Menu Item Action,Profile Banner Url,Default Profile,Default Profile Image,Geo Enabled,Language,Listed Count,Profile Background Image Url,Verified
118,elonmusk,Elon Musk,106,55096060,14297,8766,"Technoking of Tesla, Imperator of Mars 😉",NaN,NaN,2/6/2009 20:12,...,Open Twitter Page for This Person,https://twitter.com/elonmusk,https://pbs.twimg.com/profile_banners/44196397...,False,False,False,NaN,72264,https://abs.twimg.com/images/themes/theme1/bg.png,True
95,google,Google,292,23111667,133382,3150,#HeyGoogle,"Mountain View, CA",https://t.co/vXgfrAznPl,10/2/2009 19:14,...,Open Twitter Page for This Person,https://twitter.com/google,https://pbs.twimg.com/profile_banners/20536157...,False,False,True,NaN,2,https://abs.twimg.com/images/themes/theme1/bg.png,True
119,spacex,SpaceX,101,16779910,5294,246,"SpaceX designs, manufactures and launches the ...","Hawthorne, CA",https://t.co/SDnmlLwwoK,23/4/2009 21:53,...,Open Twitter Page for This Person,https://twitter.com/spacex,https://pbs.twimg.com/profile_banners/34743251...,False,False,False,NaN,31413,https://abs.twimg.com/images/themes/theme14/bg...,True
71,mlb,MLB,4044,8965803,193465,27563,Official Twitter account of Major League Baseb...,NaN,https://t.co/teGWLJu3BG,30/12/2008 15:39,...,Open Twitter Page for This Person,https://twitter.com/mlb,https://pbs.twimg.com/profile_banners/18479513...,False,False,True,NaN,31564,https://abs.twimg.com/images/themes/theme1/bg.png,True
255,gmail,Gmail,115,6297644,240655,174,"News, tips and tricks from the Gmail team. Nee...","Mountain View, CA",http://t.co/t07FedBKq3,8/5/2009 14:53,...,Open Twitter Page for This Person,https://twitter.com/gmail,https://pbs.twimg.com/profile_banners/38679388...,False,False,False,NaN,17901,https://abs.twimg.com/images/themes/theme15/bg...,True
162,el_universal_mx,El Universal,14268,6006182,1011400,31561,Cuenta oficial de El Gran Diario de México. 🎙️...,Centro de la Ciudad de México,https://t.co/3O9EUfr6WJ,10/10/2008 0:09,...,Open Twitter Page for This Person,https://twitter.com/el_universal_mx,https://pbs.twimg.com/profile_banners/16676396...,False,False,True,NaN,24679,https://abs.twimg.com/images/themes/theme1/bg.png,True
88,20thcentury,20th Century Studios,1690,4461185,8124,2343,Welcome to the Official Twitter page for 20th ...,NaN,https://t.co/KohQXsZ9wV,15/10/2010 17:31,...,Open Twitter Page for This Person,https://twitter.com/20thcentury,https://pbs.twimg.com/profile_banners/20316299...,False,False,True,NaN,6303,https://abs.twimg.com/images/themes/theme1/bg.png,True
99,googlemaps,Google Maps,58,3851972,81700,3319,Find what you need by getting the latest infor...,NaN,https://t.co/xcRgWLk1TG,15/4/2009 2:29,...,Open Twitter Page for This Person,https://twitter.com/googlemaps,https://pbs.twimg.com/profile_banners/31311757...,False,False,True,NaN,16762,https://abs.twimg.com/images/themes/theme1/bg.png,True
112,awscloud,Amazon Web Services,871,1901571,35391,2327,Official Twitter Feed for Amazon Web Services....,"Seattle, WA",https://t.co/U4UrovnUta,18/8/2009 19:52,...,Open Twitter Page for This Person,https://twitter.com/awscloud,https://pbs.twimg.com/profile_banners/66780587...,False,False,True,NaN,8617,https://abs.twimg.com/images/themes/theme1/bg.png,True
215,linkedin,LinkedIn,30,1567469,18691,5781,"With your community by your side, there’s no t...","Sunnyvale, CA",https://t.co/3gG68vGZzD,4/2/2008 18:22,...,Open Twitter Page for This Person,https://twitter.com/linkedin,https://pbs.twimg.com/profile_banners/13058772...,False,False,True,NaN,17613,https://abs.twimg.com/images/themes/theme1/bg.png,True


In [37]:
influencers.Verified.value_counts()

False    207
True      57
Name: Verified, dtype: int64